In [26]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.transforms as transforms
from model import CNN
from loss import loss_coteaching

import random
import numpy as np
from PIL import Image
import json
import os
from tqdm import tqdm
from loader_CIFAR import CifarDataloader, CifarDataset


# def unpickle(file):
#     import _pickle as cPickle
#     with open(file, 'rb') as fo:
#         dict = cPickle.load(fo, encoding='latin1')
#     return dict
#
#
# class cifarDataset(Dataset):
#     def __init__(self, dataset, root_dir, transform, mode, noise_file=''):
#
#         self.transform = transform
#         self.mode = mode
#         self.transition = {0: 0, 2: 0, 4: 7, 7: 7, 1: 1, 9: 1, 3: 5, 5: 3, 6: 6,
#                            8: 8}  # class transition for asymmetric noise for cifar10
#         # generate asymmetric noise for cifar100
#         self.transition_cifar100 = {}
#         nb_superclasses = 20
#         nb_subclasses = 5
#         base = [1, 2, 3, 4, 0]
#         for i in range(nb_superclasses * nb_subclasses):
#             self.transition_cifar100[i] = int(base[i % 5] + 5 * int(i / 5))
#
#         if self.mode == 'test':
#             if dataset == 'cifar10':
#                 test_dic = unpickle('%s/test_batch' % root_dir)
#                 self.test_data = test_dic['data']
#                 self.test_data = self.test_data.reshape((10000, 3, 32, 32))
#                 self.test_data = self.test_data.transpose((0, 2, 3, 1))
#                 self.test_label = test_dic['labels']
#             elif dataset == 'cifar100':
#                 test_dic = unpickle('%s/test' % root_dir)
#                 self.test_data = test_dic['data']
#                 self.test_data = self.test_data.reshape((10000, 3, 32, 32))
#                 self.test_data = self.test_data.transpose((0, 2, 3, 1))
#                 self.test_label = test_dic['fine_labels']
#         else:
#             train_data = []
#             train_label = []
#             if dataset == 'cifar10':
#                 for n in range(1, 6):
#                     dpath = '%s/data_batch_%d' % (root_dir, n)
#                     data_dic = unpickle(dpath)
#                     train_data.append(data_dic['data'])
#                     train_label = train_label + data_dic['labels']
#                 train_data = np.concatenate(train_data)
#             elif dataset == 'cifar100':
#                 train_dic = unpickle('%s/train' % root_dir)
#                 train_data = train_dic['data']
#                 train_label = train_dic['fine_labels']
#                 # print(train_label)
#                 # print(len(train_label))
#             train_data = train_data.reshape((50000, 3, 32, 32))
#             train_data = train_data.transpose((0, 2, 3, 1))
#
#             noise_label = json.load(open(noise_file, "r"))
#
#             if self.mode == 'train':
#                 self.train_data = train_data
#                 self.noise_label = noise_label
#                 self.clean_label = train_label
#
#     def __getitem__(self, index):
#         if self.mode == 'train':
#             img, target = self.train_data[index], self.noise_label[index]
#             img = Image.fromarray(img)
#             img = self.transform(img)
#             return img, target, index
#         elif self.mode == 'test':
#             img, target = self.test_data[index], self.test_label[index]
#             img = Image.fromarray(img)
#             img = self.transform(img)
#             return img, target
#
#     def __len__(self):
#         if self.mode != 'test':
#             return len(self.train_data)
#         else:
#             return len(self.test_data)
#
#
# class cifar_dataloader():
#     def __init__(self, dataset, batch_size, num_workers, root_dir, noise_file=''):
#         self.dataset = dataset
#         self.batch_size = batch_size
#         self.num_workers = num_workers
#         self.root_dir = root_dir
#         self.noise_file = noise_file
#         if self.dataset == 'cifar10':
#             self.transform_train = transforms.Compose([
#                 transforms.RandomCrop(32, padding=4),
#                 transforms.RandomHorizontalFlip(),
#                 transforms.ToTensor(),
#                 transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
#             ])
#             self.transform_test = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
#             ])
#         elif self.dataset == 'cifar100':
#             self.transform_train = transforms.Compose([
#                 transforms.RandomCrop(32, padding=4),
#                 transforms.RandomHorizontalFlip(),
#                 transforms.ToTensor(),
#                 transforms.Normalize((0.507, 0.487, 0.441), (0.267, 0.256, 0.276)),
#             ])
#             self.transform_test = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize((0.507, 0.487, 0.441), (0.267, 0.256, 0.276)),
#             ])
#
#     def run(self, mode):
#         if mode == 'train':
#             train_dataset = cifarDataset(dataset=self.dataset,
#                                          root_dir=self.root_dir, transform=self.transform_train, mode="train",
#                                          noise_file=self.noise_file)
#             trainloader = DataLoader(
#                 dataset=train_dataset,
#                 batch_size=self.batch_size,
#                 shuffle=True,
#                 num_workers=self.num_workers)
#             return trainloader, np.asarray(train_dataset.noise_label), np.asarray(train_dataset.clean_label)
#
#         elif mode == 'test':
#             test_dataset = cifarDataset(dataset=self.dataset,
#                                         root_dir=self.root_dir, transform=self.transform_test, mode='test')
#             test_loader = DataLoader(
#                 dataset=test_dataset,
#                 batch_size=self.batch_size,
#                 shuffle=False,
#                 num_workers=self.num_workers)
#             return test_loader


In [27]:

# test the custom loaders for CIFAR
dataset = 'cifar10'  # either cifar10 or cifar100
data_path = 'rawdata_CIFAR10'  # path to the data file (don't forget to download the feature data and also put the noisy label file under this folder)

num_iter_per_epoch = 100
num_print_freq = 100
num_epoch = 200
num_batch_size = 128
num_gradual = 10
num_exponent = 1
num_forget_rate = 0.1
num_noise_rate = 0.2
num_workers = 4
num_classes = 10
num_learning_rate = 0.001
num_epoch_decay_start = 80
num_input_channel = 3

# Adjust learning rate and betas for Adam Optimizer
mom1 = 0.9
mom2 = 0.1
alpha_plan = [num_learning_rate] * num_epoch
beta1_plan = [mom1] * num_epoch
for i in range(num_epoch_decay_start, num_epoch):
    alpha_plan[i] = float(num_epoch - i) / (num_epoch - num_epoch_decay_start) * num_learning_rate
    beta1_plan[i] = mom2

rate_schedule = np.ones(num_epoch) * num_forget_rate
rate_schedule[:num_gradual] = np.linspace(0, num_forget_rate ** num_exponent, num_gradual)

json_noise_file_names = {
    1: 'cifar10_noisy_labels_task1.json',
    2: 'cifar10_noisy_labels_task2.json',
    3: 'cifar10_noisy_labels_task3.json'
}
noise_file_name = json_noise_file_names[1]

loader = CifarDataloader(dataset, batch_size=128,
                          num_workers=10,
                          root_dir=data_path,
                          noise_file='%s/%s' % (data_path, noise_file_name))

all_trainloader, noisy_labels, clean_labels = loader.run('train')
noise_or_not = np.transpose(noisy_labels) == np.transpose(clean_labels)
test_loader = loader.run('test')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_full_name = f'comixing_{dataset}_{noise_file_name}'

In [28]:
def adjust_learning_rate(optimizer, epoch):
    for param_group in optimizer.param_groups:
        param_group['lr']=alpha_plan[epoch]
        param_group['betas']=(beta1_plan[epoch], 0.999) # Only change beta1


In [29]:
# TODO: Complete migrating the Co-Teaching model
def accuracy(logit, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    output = F.softmax(logit, dim=1)
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


def train(loader_train, epoch, model1, optimizer1, model2, optimizer2):
    print("Training... %s" % model_full_name)
    pure_ratio_list = []
    pure_ratio_1_list = []
    pure_ratio_2_list = []
    count_total_train1 = 0
    count_total_correct1 = 0
    count_total_train2 = 0
    count_total_correct2 = 0

    for i, (images, labels, indexes) in enumerate(loader_train):
        ind = indexes.cpu().numpy().transpose()
        if i > num_iter_per_epoch:
            break

        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        # Forward Backward Optimize
        output1 = model1(images)
        prec1, _ = accuracy(output1, labels, topk=(1, 5))
        count_total_train1 += 1
        count_total_correct1 += prec1

        output2 = model2(images)
        prec2, _ = accuracy(output2, labels, topk=(1, 5))
        count_total_train2 += 1
        count_total_correct2 += prec2

        loss1, loss2, pure_ratio_1, pure_ratio_2 = loss_coteaching(output1, output2, labels, rate_schedule[epoch], ind,
                                       noise_or_not)
        pure_ratio_1_list.append(100*pure_ratio_1)
        pure_ratio_2_list.append(100*pure_ratio_2)

        optimizer1.zero_grad()
        loss1.backward()
        optimizer1.step()
        optimizer2.zero_grad()
        loss2.backward()
        optimizer2.step()
        if (i + 1) % num_print_freq == 0:
            print(
                'Epoch [%d/%d], Iter [%d/%d] Training Accuracy1: %.4F, Training Accuracy2: %.4f, Loss1: %.4f, Loss2: %.4f, Pure Ratio1: %.4f, Pure Ratio2 %.4f'
                % (epoch + 1, num_epoch, i + 1, len(all_trainloader) // num_batch_size, prec1, prec2, loss1.item(),
                   loss2.item(), np.sum(pure_ratio_1_list) / len(pure_ratio_1_list),
                   np.sum(pure_ratio_2_list) / len(pure_ratio_2_list)))

    train_acc1 = float(count_total_correct1) / float(count_total_train1)
    train_acc2 = float(count_total_correct2) / float(count_total_train2)
    return train_acc1, train_acc2, pure_ratio_1_list, pure_ratio_2_list


def evaluate(test_loader, model1, model2):
    print('Evaluating %s...' % model_full_name)
    model1 = model1.to(device)  # Change model to 'eval' mode.
    model2 = model2.to(device)
    correct1 = 0
    total1 = 0
    print("Start evaluating model 1")
    for images, labels in tqdm(test_loader):
        images = Variable(images).to(device)
        labels = labels.to(device)
        logits1 = model1(images)
        outputs1 = F.softmax(logits1, dim=1)
        _, pred1 = torch.max(outputs1.data, 1)
        total1 += labels.size(0)
        correct1 += (pred1 == labels).sum()

    print("Start evaluating model 2")
    model2.eval()  # Change model to 'eval' mode
    correct2 = 0
    total2 = 0
    for images, labels in tqdm(test_loader):
        images = Variable(images).to(device)
        labels = labels.to(device)

        logits2 = model2(images)
        outputs2 = F.softmax(logits2, dim=1)
        _, pred2 = torch.max(outputs2.data, 1)
        total2 += labels.size(0)
        correct2 += (pred2 == labels).sum()

    acc1 = 100 * float(correct1) / float(total1)
    acc2 = 100 * float(correct2) / float(total2)
    return acc1, acc2


In [ ]:

# Data Loader (Input Pipeline)
print('loading dataset...')

# train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
#                                            batch_size=num_batch_size,
#                                            num_workers=num_workers,
#                                            drop_last=True,
#                                            shuffle=True)
#
# test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
#                                           batch_size=num_batch_size,
#                                           num_workers=num_workers,
#                                           drop_last=True,
#                                           shuffle=False)

train_loader = all_trainloader

# Define models
print('building model...')
cnn1 = CNN(input_channel=num_input_channel, n_outputs=num_classes)
cnn1.to(device)
# print(cnn1.parameters)
optimizer1 = torch.optim.Adam(cnn1.parameters(), lr=num_learning_rate)

cnn2 = CNN(input_channel=num_input_channel, n_outputs=num_classes)
cnn2.to(device)
# print(cnn2.parameters)
optimizer2 = torch.optim.Adam(cnn2.parameters(), lr=num_learning_rate)

mean_pure_ratio1=0
mean_pure_ratio2=0

# with open(txtfile, "a") as myfile:
#     myfile.write('epoch: train_acc1 train_acc2 test_acc1 test_acc2 pure_ratio1 pure_ratio2\n')

epoch=0
train_acc1=0
train_acc2=0
# evaluate models with random weights
test_acc1, test_acc2=evaluate(test_loader, cnn1, cnn2)
print('Epoch [%d/%d] Test Accuracy on the %s test images: Model1 %.4f %% Model2 %.4f %% Pure Ratio1 %.4f %% Pure Ratio2 %.4f %%' % (epoch+1, num_epoch, len(test_loader.dataset), test_acc1, test_acc2, mean_pure_ratio1, mean_pure_ratio2))
# save results
# with open(txtfile, "a") as myfile:
#     myfile.write(str(int(epoch)) + ': '  + str(train_acc1) +' '  + str(train_acc2) +' '  + str(test_acc1) + " " + str(test_acc2) + ' '  + str(mean_pure_ratio1) + ' '  + str(mean_pure_ratio2) + "\n")

# training
for epoch in range(1, num_epoch):
    # train models
    cnn1.train()
    adjust_learning_rate(optimizer1, epoch)
    cnn2.train()
    adjust_learning_rate(optimizer2, epoch)
    train_acc1, train_acc2, pure_ratio_1_list, pure_ratio_2_list=train(train_loader, epoch, cnn1, optimizer1, cnn2, optimizer2)
    # evaluate models
    test_acc1, test_acc2=evaluate(test_loader, cnn1, cnn2)
    # save results
    mean_pure_ratio1 = sum(pure_ratio_1_list)/len(pure_ratio_1_list)
    mean_pure_ratio2 = sum(pure_ratio_2_list)/len(pure_ratio_2_list)
    print('Epoch [%d/%d] Test Accuracy on the %s test images: Model1 %.4f %% Model2 %.4f %%, Pure Ratio 1 %.4f %%, Pure Ratio 2 %.4f %%' % (epoch+1, num_epoch, len(test_loader.dataset), test_acc1, test_acc2, mean_pure_ratio1, mean_pure_ratio2))
    # with open(txtfile, "a") as myfile:
    #     myfile.write(str(int(epoch)) + ': '  + str(train_acc1) +' '  + str(train_acc2) +' '  + str(test_acc1) + " " + str(test_acc2) + ' ' + str(mean_pure_ratio1) + ' ' + str(mean_pure_ratio2) + "\n")

  0%|          | 0/79 [00:00<?, ?it/s]

loading dataset...
building model...
Evaluating comixing_cifar10_cifar10_noisy_labels_task1.json...
Start evaluating model 1


  0%|          | 0/79 [00:00<?, ?it/s]

Start evaluating model 2


100%|██████████| 79/79 [00:13<00:00,  6.05it/s]


...

Start evaluating model 2


100%|██████████| 79/79 [00:13<00:00,  5.97it/s]


Epoch [8/200] Test Accuracy on the 10000 test images: Model1 51.2900 % Model2 51.3200 %, Pure Ratio 1 68.9210 %, Pure Ratio 2 68.9042 %
Training... comixing_cifar10_cifar10_noisy_labels_task1.json
Epoch [9/200], Iter [100/3] Training Accuracy1: 32.8125, Training Accuracy2: 37.5000, Loss1: 0.0148, Loss2: 0.0148, Pure Ratio1: 69.6724, Pure Ratio2 69.7500


  0%|          | 0/79 [00:00<?, ?it/s]

Evaluating comixing_cifar10_cifar10_noisy_labels_task1.json...
Start evaluating model 1
